# **SIIM**


The Society for Imaging Informatics in Medicine (SIIM) is the leading healthcare professional organization for those interested in the current and future use of informatics in medical imaging. The society's mission is to advance medical imaging informatics across the enterprise through education, research, and innovation in a multi-disciplinary community

![](https://drive.google.com/uc?id=1Yhk63WM5MCTj0KLhhdGeiXsp8oaSxhTK)

Image Source : siim.org

# **ISIC :**

The International Skin Imaging Collaboration: Melanoma Project is an academia and industry partnership designed to facilitate the application of digital skin imaging to help reduce melanoma mortality. When recognized and treated in its earliest stages, melanoma is readily curable. Digital images of skin lesions can be used to educate professionals and the public in melanoma recognition as well as directly aid in the diagnosis of melanoma through teledermatology, clinical decision support, and automated diagnosis.


# **Skin Cancer :**

Skin cancer is the most prevalent type of cancer. Melanoma, specifically, is responsible for 75% of skin cancer deaths, despite being the least common skin cancer. The American Cancer Society estimates over 100,000 new melanoma cases will be diagnosed in 2020. It's also expected that almost 7,000 people will die from the disease. As with other cancers, early and accurate detection—potentially aided by data science—can make treatment more effective.


# **Problem Statement :**

In this workshop , you’ll identify melanoma in images of skin lesions and you’ll use images within the same patient and determine which are likely to represent a melanoma. Using patient-level contextual information may help the development of image analysis tools, which could better support clinical dermatologists.


![](https://drive.google.com/uc?id=11uCu4Bl5YHEi_xcfZ1Q86W3sOhJ6tvLy)

# **Installing the Libraries**

In [ ]:
!pip install -q --upgrade pip
!pip install -q efficientnet

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from functools import partial
import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split

# TPU


![](https://drive.google.com/uc?id=1q6AUi9XZRRWBjov49PSl3thB9idGsUKV)

Tensor Processing Units (TPUs) are Google's custom-developed application-specific integrated circuits (ASICs) used to accelerate machine learning workloads.It's easy to run replicated models on Cloud TPU using High-level Tensorflow APIs .


![](https://drive.google.com/uc?id=1-EAdsgcuVsFKO1ru00TTrKePRnv-6FHv)

# Performance : 

TPU can achieve a high computational throughput on massive multiplications and additions for neural networks , at blazingly fast speeds with much less power consumption and smaller footprint.


![](https://drive.google.com/uc?id=18f8gdJOsNad_ZBESxhxxo2SVIf-IaYGR)

📌 **TPU Initialization:** TPUs  are usually on Cloud TPU workers and hence have to be connected to remote clusters and then initialized .

📌 **Distribution strategies :** A distribution strategy is an abstraction that can be used to drive models on CPU, GPUs or TPUs. 

Credit : The code has been adapted from @mgornergooglestater kernel on Flower classification on TPUs

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

Device: grpc://10.0.0.3:8470
Number of replicas: 32


<div class="alert alert-block alert-info">
<b>Note:</b> TPUs read data directly from <code>Google Cloud Storage (GCS)</code> This Kaggle utility will copy the dataset to a GCS bucket co-located with the TPU. If you have multiple datasets attached to the notebook, you can pass the name of a specific dataset to the <code>get_gcs_path function</code>. The name of the dataset is the name of the directory it is mounted in. Use <code>!ls /kaggle/input/</code> to list attached datasets.
</div>

![](https://drive.google.com/uc?id=15QPJMCidq3gQM8oCZ6NNlRQHnImV28wP)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = KaggleDatasets().get_gcs_path('siim-isic-melanoma-classification')
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IMAGE_SIZE = [1024, 1024]

# TF Records :

📌TF-Record is Tensorflow’s own binary storage format.

📌Any byte-string that can be decoded in TensorFlow could be stored in a TFRecord file. Examples include: Lines of text, JSON ,encoded image data, or serialized tf.Tensors .

📌A TFRecord file contains a sequence of records and can only be read sequentially.Each record contains a byte-string, for the data-payload, plus the data-length, and CRC32C (32-bit CRC using the Castagnoli polynomial) hashes for integrity checking.


![](https://drive.google.com/uc?id=10r4NCvxUdK6IhEFWI1UxwVUY4PYYMMVG)

In [ ]:
TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(
    tf.io.gfile.glob(GCS_PATH + '/tfrecords/train*.tfrec'),
    test_size=0.1, random_state=42
)
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/tfrecords/test*.tfrec')
print('Train TFRecord Files:', len(TRAINING_FILENAMES))
print('Validation TFRecord Files:', len(VALID_FILENAMES))
print('Test TFRecord Files:', len(TEST_FILENAMES))

Train TFRecord Files: 14
Validation TFRecord Files: 2
Test TFRecord Files: 16


![](https://drive.google.com/uc?id=1qomd1gdCbw7CA-vdlqtBWAYqsU6-aCtN)

Image Source : mc.ai

# TF-Data

![](https://drive.google.com/uc?id=10M07nrMGrKKVP0KGwAqsm7oNK1mXexsD)

📌The tf.data API enables you to build complex input pipelines from simple, reusable pieces. The tf.data API also makes it possible to handle large amounts of data, read from different data formats, and perform complex transformations.

📌The tf.data API introduces a tf.data.Dataset abstraction that represents a sequence of elements, in which each element consists of one or more components. For example, in an image pipeline, an element might be a single training example, with a pair of tensor components representing the image and its label.

🎯 **Input Pipeline :**

 tf.data.TFRecordDataset() is used to create an input pipeline for data stored in TFRecord format .
 
🎯 **Transformations :**

The Dataset object can be transformed into a new Dataset by chaining method calls on the tf.data.Dataset object . Some of the transformations which can be applied are Dataset.map() , Dataset.batch() , Dataset.shuffle() , Dataset.prefetch() .

The Dataset object is a Python iterable which  it possible to consume its elements using a for loop .


![](https://drive.google.com/uc?id=1CLfEMEn9uiPGjhgVZc620djKmBQrwSZV)

![](https://drive.google.com/uc?id=1x383ghyybTV0jQqBSlHDEc8FWD8AqaHz)


In [ ]:
def decode_to_tensor(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_to_tensor(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

# Data Augmentation :

🎯  Data augmentation is a technique through which one can increase the size of the data for the training of the model without adding the new data. Techniques like padding, cropping, rotating, and flipping are the most common methods that are used over the images to increase the data size. 

<div class="alert alert-block alert-info">
<b>Note:</b> Both <code>Keras Preprocessing Layers</code> and <code>tf.image</code> can be used for data augmentation. For finer control , <code>tf.image</code> is preferred.
</div>


![](https://drive.google.com/uc?id=1_uGNygnCZ--G_hOP7zEr_oscGjvnBnob)

Image Source : medium.com

In [ ]:
def augmentation_pipeline(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

In [ ]:
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(augmentation_pipeline, num_parallel_calls=AUTOTUNE)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALID_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
print(
    'Dataset: {} training images, {} validation images, {} unlabeled test images'.format(
        NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES
    )
)

Dataset: 28984 training images, 4142 validation images, 10982 unlabeled test images


📌 **Cyclic learning rates :**

<div class="alert alert-block alert-info">
Cyclic learning rates  is a learning rate scheduling technique for (1) faster training of a network and (2) a finer understanding of the optimal learning rate. Cyclic learning rates have an effect on the model training process known somewhat fancifully as<code>"superconvergence"</code> 
</div>

Source : @residentmario Notebook

![](https://drive.google.com/uc?id=1bnsglJByd1lc1h9hGrz2WZa1LzVBgtWW)

Image Source : https://arxiv.org/abs/1803.09820

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, lr_min=0.000001, lr_rampup_epochs=20, lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync
    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay ** (epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

# EfficientNets

**Compound Model Scaling: A Better Way to Scale Up CNNs**

EfficientNets uses a new scaling method that uniformly scales all dimensions of depth/width/resolution using a simple yet highly effective compound coefficient The first step in the compound scaling method is to perform a grid search to find the relationship between different scaling dimensions of the baseline network under a fixed resource constraint.This determines the appropriate scaling coefficient for each of the dimensions mentioned above. We then apply those coefficients to scale up the baseline network to the desired target model size or computational budget.

This compound scaling method consistently improves model accuracy and efficiency for scaling up existing models such as MobileNet (+1.4% imagenet accuracy), and ResNet (+0.7%), compared to conventional scaling methods.



![](https://drive.google.com/uc?id=18ZzZw0U-4O4iPe1JBVZwUVcn4FCmTS_x)

**EfficientNet Architecture :**

The effectiveness of model scaling also relies heavily on the baseline network which  is developed by performing a neural architecture search using the AutoML MNAS framework, which optimizes both accuracy and efficiency (FLOPS). The resulting architecture uses mobile inverted bottleneck convolution (MBConv), similar to MobileNetV2 and MnasNet, but is slightly larger due to an increased FLOP budget.The baseline network is then scaled up to obtain a family of models, called EfficientNets.

![](https://drive.google.com/uc?id=1KVGvxtN4XXvLbt3hCSSZBU04lv0LqgO-)

📌 **EfficientNet Performance**

<div class="alert alert-block alert-info">
<b>Note:</b>
The EfficientNet models achieve both higher accuracy and better efficiency over existing CNNs like ResNet , DenseNet etc , reducing parameter size and FLOPS by an order of magnitude. <code></code> 
</div>


![](https://drive.google.com/uc?id=1jYu8AMPI9LVF2TQKUgJyKZaZqpsm3U0k)

Image Source : EfficientNet Research Paper

In [ ]:
with strategy.scope():
    
    train_dataset = get_training_dataset()
    valid_dataset = get_validation_dataset()
    
    model = tf.keras.Sequential([
        efn.EfficientNetB6(
            input_shape=(*IMAGE_SIZE, 3),
            weights='imagenet',
            include_top=False
        ),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation= 'relu'), 
        tf.keras.layers.Dropout(0.25), 
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy']
    )

model.summary()

165527552/165527152 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b6 (Model)      (None, 32, 32, 2304)      40960136  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1180160   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 42,140,809
Trainable params: 41,916,377
Non-trainable params: 224,432
_________________________________________________________________


In [ ]:
lrfn = build_lrfn()
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALID_STEPS = NUM_VALIDATION_IMAGES // BATCH_SIZE


history = model.fit(
    train_dataset, epochs=1,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_dataset,
    validation_steps=VALID_STEPS,
    callbacks=[
        tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join("./model.h5"),
            monitor='train_loss', verbose=0,
            save_best_only=True, save_weights_only=False,
            mode='auto', save_freq='epoch'
        )
    ]
)

In [ ]:
sub = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')
sub.head()

,image_name,target
0,ISIC_0052060,0
1,ISIC_0052349,0
2,ISIC_0058510,0
3,ISIC_0073313,0
4,ISIC_0073502,0


In [ ]:
test_ds = get_test_dataset(ordered=True)

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)

Computing predictions...


In [ ]:
print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

Generating submission.csv file...


In [ ]:
pred_df = pd.DataFrame({'image_name': test_ids, 'target': np.concatenate(probabilities)})
pred_df.head()

,image_name,target
0,ISIC_6381819,0.524593
1,ISIC_5583376,0.516386
2,ISIC_6408546,0.497855
3,ISIC_6932354,0.494086
4,ISIC_8191278,0.459409


In [ ]:
del sub['target']
sub = sub.merge(pred_df, on='image_name')
sub.to_csv('submission.csv', index=False)
sub.head()

,image_name,target
0,ISIC_0052060,0.503272
1,ISIC_0052349,0.494390
2,ISIC_0058510,0.500467
3,ISIC_0073313,0.486549
4,ISIC_0073502,0.530081


Follow me on Twitter @Urengaraju

# References :

https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu

https://www.tensorflow.org/tutorials/load_data/tfrecord

https://cloud.google.com/tpu

https://www.tensorflow.org/guide/tpu

https://www.tensorflow.org/tutorials/images/data_augmentation

https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html

